pip install bvhio --upgrade


# README

In [1]:
import bvhio

# Reads the file into a deserialized tree structure.
bvh = bvhio.readAsBvh('bvhio/tests/example.bvh')
print(f'Root: {bvh.Root}')
print(f'Frames: {bvh.FrameCount}')
print(f'Frame time: {bvh.FrameTime}')

# Properties of joints in the bvh tree structure.
bvh.Root.Name
bvh.Root.Offset
bvh.Root.Channels
bvh.Root.EndSite
bvh.Root.Keyframes
bvh.Root.Children

# Calculated properties that depend on the hierarchy.
bvh.Root.getRotation()
bvh.Root.getLength()
bvh.Root.getTip()
pass

Root: Hips
Frames: 2
Frame time: 0.033333


In [2]:
import bvhio

# Reads the file into a transform tree structure and converts all data to build proper local and world spaces.
# This structure allows for extensive read of properties and spaces and does also support modifications of the animation.
root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')
root.printTree()

# load rest pose and print data
root.loadRestPose(recursive=True)
print('\nRest pose position and Y-direction of each joint in world space ')
for joint, index, depth in root.layout():
    print(f'{joint.PositionWorld} {joint.UpWorld} {joint.Name}')

Hips
+- Chest
|  +- Neck
|  |  +- Head
|  +- LeftCollar
|  |  +- LeftUpArm
|  |     +- LeftLowArm
|  |        +- LeftHand
|  +- RightCollar
|     +- RightUpArm
|        +- RightLowArm
|           +- RightHand
+- LeftUpLeg
|  +- LeftLowLeg
|     +- LeftFoot
+- RightUpLeg
   +- RightLowLeg
      +- RightFoot

Rest pose position and Y-direction of each joint in world space 
vec3(            0,            0,            0 ) vec3(            0,            1,            0 ) Hips
vec3(            0,         5.21,            0 ) vec3(            0,     0.997333,    0.0729792 ) Chest
vec3(            0,        23.86,  1.19209e-07 ) vec3(            0,            1,            0 ) Neck
vec3(            0,        29.31,  1.19209e-07 ) vec3(            0,            1,            0 ) Head
vec3(         1.12,        21.44,         1.87 ) vec3(            1,  5.96046e-08,            0 ) LeftCollar
vec3(         6.66,        21.44,         1.87 ) vec3(            0,           -1,            0 ) LeftUp

In [3]:
import bvhio

# The package allows to make modifcation on the animation data very conviniently.
root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')

# Add a root bone to the hierarchy and set itself as 'root'.
root = bvhio.Joint('Root').attach(root)

# Scale so the data represent roughly meters, assuming the data is in inches.
# Because the scale is on the root and the rest pose, it is applied to all world space data.
# Keep in mind that local keyframe and child rest pose data is still in inches!
# root.RestPose.Scale = 0.0254

# Set all joints to the first keyframe.
# The animation pose is calculated by -> Pose = RestPose + Keyframe.
root.loadPose(0)

# print info
print('\nPosition, Y-direction and local scale of each joint in world space ')
for joint, index, depth in root.layout():
    print(f'{joint.PositionWorld} {joint.UpWorld} {joint.Scale} {joint.Name}')


Position, Y-direction and local scale of each joint in world space 
vec3(            0,            0,            0 ) vec3(            0,            1,            0 ) vec3(            1,            1,            1 ) Root
vec3(         8.03,        35.01,        88.36 ) vec3(    0.0575125,     0.965201,     0.255108 ) vec3(            1,            1,            1 ) Hips
vec3(      8.32964,      40.0387,      89.6891 ) vec3(    0.0481175,     0.852046,    -0.521251 ) vec3(            1,            1,            1 ) Chest
vec3(      9.16411,       56.599,      81.1521 ) vec3(     0.163858,     0.314978,    -0.934847 ) vec3(            1,            1,            1 ) Neck
vec3(      10.0571,      58.3157,      76.0571 ) vec3(     0.136333,     0.863658,    -0.485292 ) vec3(            1,            1,            1 ) Head
vec3(      8.02774,      53.6106,      80.5308 ) vec3(    -0.967669,     0.251636,    0.0172419 ) vec3(            1,            1,            1 ) LeftCollar
vec3(      2

In [4]:
import bvhio

root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')
root = bvhio.Joint('Root', restPose=bvhio.Pose(scale=2.54)).attach(root)

# Load poses, then extract from all joints their position in world space.
pose0positions = [joint.PositionWorld for (joint, index, depth) in root.loadPose(0).layout()]
pose1positions = [joint.PositionWorld for (joint, index, depth) in root.loadPose(1).layout()]

print('Change in positions in centimeters between frame 0 and 1:')
for (joint, index, depth) in root.layout():
    print(f'{pose1positions[index] - pose0positions[index]} {joint.Name}')

Change in positions in centimeters between frame 0 and 1:
vec3(            0,            0,            0 ) Root
vec3(    -0.558798,       0.2286,      -4.8006 ) Hips
vec3(    -0.469618,     0.324989,     -5.21318 ) Chest
vec3(     0.296169,     -1.28726,     -8.01178 ) Neck
vec3(     0.350996,     -2.16693,     -8.26605 ) Head
vec3(     0.161901,     -1.32426,     -7.49876 ) LeftCollar
vec3(     0.790674,  0.000457764,     -10.1185 ) LeftUpArm
vec3(      1.05577,      3.92007,     -8.26009 ) LeftLowArm
vec3(     0.350739,      5.54311,     -9.92455 ) LeftHand
vec3(     0.158957,     -1.40071,     -7.48221 ) RightCollar
vec3(    -0.532215,     -1.32915,     -4.50168 ) RightUpArm
vec3(    -0.389343,     -4.22717,     -4.83904 ) RightLowArm
vec3(      2.33563,     -6.29668,     -6.45831 ) RightHand
vec3(    -0.659554,     0.479576,     -5.20877 ) LeftUpLeg
vec3(     -2.30601,    0.0990372,     -10.0871 ) LeftLowLeg
vec3(      1.48517,     0.579193,     -9.71515 ) LeftFoot
vec3(     -0.458

In [5]:
import bvhio

# load data as simple deserialized structure
bvhRoot = bvhio.readAsBvh('bvhio/tests/example.bvh').Root

# convert to a joint hierarchy
hierarchyRoot = bvhio.convertBvhToHierarchy(bvhRoot)

# convert them back to simple deserialized structure.
# the frame count needs to be given, and the max frame id is selected.
bvhRoot = bvhio.convertHierarchyToBvh(hierarchyRoot, hierarchyRoot.getKeyframeRange()[1] + 1)

# writes the data back into a .bvh file
bvhio.writeBvh('test.bvh', bvhio.BvhContainer(bvhRoot, len(bvhRoot.Keyframes), 1/30))

In [6]:
import bvhio

# load data
root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')

# scales the frame id of the two given frames.
# this will restult in the ids 0 and 100.
# frames without keyframe are linearly interpolated.
for joint, index, depth in root.layout():
    joint.Keyframes = [(frame * 100, key) for frame, key in joint.Keyframes]

# persists the interpolations automatically
bvhio.writeHierarchy('test.bvh', root, 1/30)

In [7]:
import bvhio

# create custom hierarchy.
root = bvhio.Joint('Root', (0,2,0)).setEuler((0,0,0)).attach(
    bvhio.Joint('UpperLegL', (+.3,2.1,0)).setEuler((0,0,180)).attach(
        bvhio.Joint('LowerLegL', (+.3,1,0)).setEuler((0,0,180))
    ),
    bvhio.Joint('UpperLegR', (-.3,2.1,0)).setEuler((0,0,180)).attach(
        bvhio.Joint('LowerLegR', (-.3,1,0)).setEuler((0,0,180))
    ),
)

# sets current layout as rest pose
root.writeRestPose(recursive=True)

# change the pose of the hierarchy to save it alter as key frame
# this will add a rotation to each leg joint for left and right side
for joint in root.filter('LegL'):
    joint.Rotation *= bvhio.Euler.toQuatFrom((+0.523599,0,0))
for joint in root.filter('LegR'):
    joint.Rotation *= bvhio.Euler.toQuatFrom((-0.523599,0,0))

# Persists the current pose as pose.
# This will calculate the keyframe differences to the rest pose.
root.writePose(0, recursive=True)

# The rest pose is loaded first to have the base pose again, this is not necessary.
root.loadRestPose(recursive=True)

# Now the same thing again with other rotations two have two keyframes.
for joint in root.filter('LegL'):
    joint.Rotation *= bvhio.Euler.toQuatFrom((-0.523599,0,0))
for joint in root.filter('LegR'):
    joint.Rotation *= bvhio.Euler.toQuatFrom((+0.523599,0,0))

# persists the current pose again as new pose.
# All keyframes between the first and this pose are linearly interpolated.
root.writePose(20, recursive=True)

# store the animation
bvhio.writeHierarchy('test.bvh', root, 1/30)

# TESTS

In [8]:
import bvhio
root = bvhio.readAsBvh('bvhio/tests/example.bvh').Root

for j, i, d in root.layout(): print(f"( {d}, glm.{j.Offset}, {j.Channels}, {len(j.Children)}, glm.{j.EndSite}, glm.{j.getTip()}, {j.getLength():09.6f}, glm.{j.getRotation()}, '{j .Name}', ),")
for j, i, d in root.layout():
    print(f'[ ', end='')
    for p in j.Keyframes:
        print(f'[ glm.{p.Position}, glm.{p.Rotation} ], ', end='')
    print(f' ],')

( 0, glm.vec3(            0,            0,            0 ), ['Xposition', 'Yposition', 'Zposition', 'Zrotation', 'Xrotation', 'Yrotation'], 3, glm.vec3(            0,            1,            0 ), glm.vec3(            0,      1.73667,            0 ), 01.736667, glm.quat(            1,            0,            0,            0 ), 'Hips', ),
( 1, glm.vec3(            0,         5.21,            0 ), ['Zrotation', 'Xrotation', 'Yrotation'], 3, glm.vec3(            0,            1,            0 ), glm.vec3(            0,      17.0367,      1.24667 ), 17.082216, glm.quat(     0.999333,    0.0365139,            0,            0 ), 'Chest', ),
( 2, glm.vec3(            0,        18.65,            0 ), ['Zrotation', 'Xrotation', 'Yrotation'], 1, glm.vec3(            0,            1,            0 ), glm.vec3(            0,         5.45,            0 ), 05.450000, glm.quat(            1,            0,            0,            0 ), 'Neck', ),
( 3, glm.vec3(            0,         5.45,            0 )

In [3]:
import bvhio
root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')
# root = bvhio.readAsHierarchy('E:\SYNC\Studium\Master\Thesis\MoCap\Bandai\dataset\Bandai-Namco-Research-Motiondataset-1\data\dataset-1_walk_normal_001.bvh')

root.loadRestPose(recursive=True)
# for j, i, d in root.layout(): print(f"( {d}, {len(j.Children)}, {len(j.Keyframes)}, {j.KeyframeRange}, '{j.Name}', ),")
# for j, i, d in root.layout(): print(f"( glm.{j.PositionLocal}, glm.{j.RotationLocal}, glm.{j.ScaleLocal}, ),")
for j, i, d in root.layout(): print(f"( glm.{j.PositionWorld}, glm.{j.RotationWorld}, glm.{j.ScaleWorld}, ),")
# for j, i, d in root.layout(): print(f"{j.PositionWorld}{j.RightWorld}{j.UpWorld}{j.Name}")

# root.loadPose(0, recursive=True)
# # for j, i, d in root.layout(): print(f"( glm.{j.PositionLocal}, glm.{j.RotationLocal}, glm.{j.ScaleLocal}, ),")
# # for j, i, d in root.layout(): print(f"( glm.{j.PositionWorld}, glm.{j.RotationWorld}, glm.{j.ScaleLocal}, ),")
# for j, i, d in root.layout(): print(f"{j.PositionWorld}{j.RightWorld}{j.UpWorld}{j.Name}")

( glm.vec3(            0,            0,            0 ), glm.quat(            1,            0,            0,            0 ), glm.vec3(            1,            1,            1 ), ),
( glm.vec3(            0,         5.21,            0 ), glm.quat(     0.999333,    0.0365139,            0,            0 ), glm.vec3(            1,            1,            1 ), ),
( glm.vec3(            0,        23.86,  1.19209e-07 ), glm.quat(            1,            0,            0,            0 ), glm.vec3(            1,            1,            1 ), ),
( glm.vec3(            0,        29.31,  1.19209e-07 ), glm.quat(            1,            0,            0,            0 ), glm.vec3(            1,            1,            1 ), ),
( glm.vec3(         1.12,        21.44,         1.87 ), glm.quat(     0.707107,            0,            0,    -0.707107 ), glm.vec3(            1,            1,            1 ), ),
( glm.vec3(         6.66,        21.44,         1.87 ), glm.quat(            0,            0,  

In [1]:
import bvhio
import glm

root = bvhio.readAsHierarchy('C:/Users/ericd/Documents/Thesis/MoCap/Bandai/dataset/Bandai-Namco-Research-Motiondataset-1/data/dataset-1_walk_normal_001.bvh')
layout = root.layout()

root.loadRestPose()
layout[ 0][0].setEuler((   0,   0,   0))                # joint_Root
layout[ 1][0].setEuler((   0,   0,   0))                # Hips
layout[ 2][0].setEuler((   0,   0,   0))                # Spine
layout[ 3][0].setEuler((   0, +90,   0)).roll(-90)      # Chest
layout[ 4][0].setEuler((   0,   0,   0))                # Neck
layout[ 5][0].setEuler((   0,   0,   0))                # Head

layout[ 6][0].setEuler((   0,   0, -90))                # Shoulder_L
layout[ 7][0].setEuler((   0,   0, -90))                # UpperArm_L
layout[ 8][0].setEuler((   0,   0,   0))                # LowerArm_L
layout[ 9][0].setEuler((   0,   0,   0))                # Hand_L

layout[10][0].setEuler((   0,   0, +90))                # Shoulder_R
layout[11][0].setEuler((   0,   0, +90))                # UpperArm_R
layout[12][0].setEuler((   0,   0,   0))                # LowerArm_R
layout[13][0].setEuler((   0,   0,   0))                # Hand_R

layout[14][0].setEuler((   0,   0, 180))                # UpperLeg_L
layout[15][0].setEuler((   0,   0,   0))                # LowerLeg_L
layout[16][0].setEuler((   0,   0,   0))                # Foot_L
layout[17][0].setEuler((   0,   0,   0))                # Toes_L

layout[18][0].setEuler((   0,   0, 180))                # UpperLeg_R
layout[19][0].setEuler((   0,   0,   0))                # LowerLeg_R
layout[20][0].setEuler((   0,   0,   0))                # Foot_R
layout[21][0].setEuler((   0,   0,   0))                # Toes_R
root.writeRestPose(recursive=True, keep=['position', 'rotation', 'scale'])

for frame in range(*root.getKeyframeRange()):
    root.loadPose(frame, recursive=True)
    layout[ 2][0].roll(-90)                                   # Spine
    layout[ 3][0].roll(-90)                                   # Chest
    layout[ 4][0].roll(-90)                                   # Neck
    layout[ 5][0].roll(-90)                                   # Head
    layout[10][0].roll(180, recursive=True)                   # Shoulder_R
    layout[18][0].roll(180, recursive=True)                   # UpperLeg_R

    layout[ 5][0].Rotation *= glm.angleAxis(glm.radians(-90), (1,0,0))  # Head
    layout[ 9][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Hand_L
    layout[13][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Hand_R
    layout[17][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Toes_L
    layout[21][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Toes_R
    root.writePose(frame, recursive=True)

root.RestPose.Scale = 0.012
root.applyRestposeScale(recursive=True)


bvhio.writeHierarchy('test.bvh', root, 1/30, percision=5)

root.loadPose(0)
for j, i, d in root.layout(): print(f"{j.PositionWorld}{j.RestPose.Position}{j.RestPose.Scale}{j.Keyframes[0][1].Position}{j.Keyframes[0][1].Scale}{j.Name}")

# vec3(            0,            0,            0 )vec3(            0,            0,            0 )vec3(            0,            0,            0 )joint_Root
# vec3(   -0.0205732,      1.08324,     -4.88548 )vec3(            0,       93.972, -0.000101328 )vec3(     -1.71443,      9.24911,     -404.034 )Hips
# vec3(  -0.00892087,      1.27168,     -4.88174 )vec3( -1.90735e-06,      15.7357,            0 )vec3( -5.12077e-06,            0, -5.51764e-07 )Spine
# vec3(  -0.00391087,      1.37948,     -4.88067 )vec3( -9.53674e-07,      8.99373,            0 )vec3(  5.21684e-07,            0,  9.23889e-07 )Chest
# vec3( -0.000942838,      1.57598,     -4.88238 )vec3(  1.49012e-08,      16.3762,     0.187639 )vec3( -7.00355e-07,            0, -1.38581e-06 )Neck
# vec3(   0.00148127,       1.6528,     -4.89213 )vec3( -9.53674e-07,      6.45672,            0 )vec3(  8.51502e-07,            0, -3.04696e-07 )Head
# vec3(    0.0575751,      1.53441,     -4.89113 )vec3(      4.98039,      12.9961,    -0.093822 )vec3(  2.86102e-06,  4.76837e-07, -4.76837e-07 )Shoulder_L
# vec3(     0.121539,      1.53157,     -4.89552 )vec3( -4.76837e-07,      5.34809,            0 )vec3(            0,  7.22636e-06,  -6.2814e-07 )UpperArm_L
# vec3(     0.175011,      1.25344,     -4.96634 )vec3( -1.90735e-06,      24.3286,            0 )vec3( -1.36424e-12, -1.90735e-06,  4.90207e-06 )LowerArm_L
# vec3(     0.256975,     0.998882,     -4.97267 )vec3( -1.90735e-06,      22.2916,            0 )vec3(  9.88446e-07,            0,  4.63217e-06 )Hand_L
# vec3(   -0.0612926,      1.53632,     -4.87872 )vec3(     -4.98039,      12.9964,   -0.0938182 )vec3( -2.86102e-06,  4.76837e-07,  4.76837e-07 )Shoulder_R
# vec3(    -0.124201,      1.53249,     -4.86661 )vec3(  9.53674e-07,      5.34813,            0 )vec3(            0,  6.47881e-07,  3.79883e-06 )UpperArm_R
# vec3(    -0.215351,      1.25562,     -4.88287 )vec3(  1.90735e-06,      24.3289,            0 )vec3(  -3.8147e-06,            0,  8.65921e-06 )LowerArm_R
# vec3(    -0.300361,      1.01304,      -4.8088 )vec3(            0,      22.2915,            0 )vec3( -3.91602e-13,            0,  4.47941e-06 )Hand_R
# vec3(    0.0552594,      1.05224,     -4.88822 )vec3(      6.46944,     -2.19248,            0 )vec3(  4.76837e-07,            0, -1.22533e-07 )UpperLeg_L
# vec3(    0.0469132,     0.619101,     -4.70863 )vec3( -7.62939e-06,      39.0811,  9.09495e-13 )vec3(  6.73843e-06,            0, -3.68639e-12 )LowerLeg_L
# vec3(    0.0517683,     0.301875,     -5.08444 )vec3(  -3.8147e-06,      40.9854,  9.09495e-13 )vec3(  6.48686e-06,            0, -2.00803e-12 )Foot_L
# vec3(    0.0736028,     0.163525,     -5.10112 )vec3( -1.90735e-06,      11.7543,            0 )vec3(  2.30145e-07,            0,  3.05095e-06 )Toes_L
# vec3(   -0.0996528,      1.06174,     -4.88377 )vec3(     -6.46944,     -2.19251,            0 )vec3(  4.76837e-07,            0, -1.45428e-06 )UpperLeg_R
# vec3(    -0.043743,     0.606222,     -4.78724 )vec3(            0,      39.0811, -3.72529e-09 )vec3( -3.83213e-12,            0,  1.11759e-08 )LowerLeg_R
# vec3(   0.00570805,     0.140777,     -4.93826 )vec3(   3.8147e-06,      40.9854,            0 )vec3( -1.53556e-05,            0, -5.99936e-06 )Foot_R
# vec3(   -0.0157643,     0.051089,     -4.83154 )vec3(  1.90735e-06,      11.7543,            0 )vec3(   1.8472e-06,            0,  5.58687e-06 )Toes_R

vec3(            0,            0,            0 )vec3(            0,            0,            0 )vec3(            1,            1,            1 )vec3(            0,            0,            0 )vec3(            1,            1,            1 )joint_Root
vec3(      4.63193,      1.28376,      1.39245 )vec3(   1.1601e-06,      1.12766,  3.64236e-07 )vec3(            1,            1,            1 )vec3(      4.63193,      0.11099,      1.43272 )vec3(            1,            1,            1 )Hips
vec3(      4.64358,      1.47219,      1.39618 )vec3(  6.85621e-09,     0.188828, -2.18372e-08 )vec3(            1,            1,            1 )vec3(  2.47244e-08,  2.98023e-08, -5.66441e-08 )vec3(            1,            1,            1 )Spine
vec3(      4.64859,      1.57999,      1.39725 )vec3(  3.42811e-09,     0.107925, -1.09186e-08 )vec3(            1,            1,            1 )vec3( -1.24528e-08,  1.49012e-08,   2.6517e-09 )vec3(            1,            1,            1 )Chest
vec3(      4

In [3]:
import glm
import bvhio

# root = bvhio.readAsHierarchy('bvhio/tests/example.bvh')
root = bvhio.readAsHierarchy('C:/Users/ericd/Documents/Thesis/MoCap/Bandai/dataset/Bandai-Namco-Research-Motiondataset-1/data/dataset-1_walk_normal_001.bvh')
layout = root.layout()

root.loadRestPose()
layout[ 0][0].setEuler((   0,   0,   0))                # joint_Root
layout[ 1][0].setEuler((   0,   0,   0))                # Hips
layout[ 1][0].Position = (0, 94, 0)                     # Hips
layout[ 2][0].setEuler((   0,   0,   0))                # Spine
layout[ 3][0].setEuler((   0, +90,   0)).roll(-90)      # Chest
layout[ 4][0].setEuler((   0,   0,   0))                # Neck
layout[ 5][0].setEuler((   0,   0,   0))                # Head

layout[ 6][0].setEuler((   0,   0, -90))                # Shoulder_L
layout[ 7][0].setEuler((   0,   0, -90))                # UpperArm_L
layout[ 8][0].setEuler((   0,   0,   0))                # LowerArm_L
layout[ 9][0].setEuler((   0,   0,   0))                # Hand_L

layout[10][0].setEuler((   0,   0, +90))                # Shoulder_R
layout[11][0].setEuler((   0,   0, +90))                # UpperArm_R
layout[12][0].setEuler((   0,   0,   0))                # LowerArm_R
layout[13][0].setEuler((   0,   0,   0))                # Hand_R

layout[14][0].setEuler((   0,   0, 180))                # UpperLeg_L
layout[15][0].setEuler((   0,   0,   0))                # LowerLeg_L
layout[16][0].setEuler((   0,   0,   0))                # Foot_L
layout[17][0].setEuler((   0,   0,   0))                # Toes_L

layout[18][0].setEuler((   0,   0, 180))                # UpperLeg_R
layout[19][0].setEuler((   0,   0,   0))                # LowerLeg_R
layout[20][0].setEuler((   0,   0,   0))                # Foot_R
layout[21][0].setEuler((   0,   0,   0))                # Toes_R
root.writeRestPose(recursive=True, keep=['position', 'rotation', 'scale'])

root.RestPose.Scale = 0.012
root.applyRestposeScale(recursive=True, bakeKeyframes=True)

for frame in range(*root.getKeyframeRange()):
    root.loadPose(frame, recursive=True)
    layout[ 2][0].roll(-90)                                   # Spine
    layout[ 3][0].roll(-90)                                   # Chest
    layout[ 4][0].roll(-90)                                   # Neck
    layout[ 5][0].roll(-90)                                   # Head
    layout[10][0].roll(180, recursive=True)                   # Shoulder_R
    layout[18][0].roll(180, recursive=True)                   # UpperLeg_R

    layout[ 5][0].Rotation *= glm.angleAxis(glm.radians(-90), (1,0,0))  # Head
    layout[ 9][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Hand_L
    layout[13][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Hand_R
    layout[17][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Toes_L
    layout[21][0].Rotation *= glm.angleAxis(glm.radians(-90), (0,0,1))  # Toes_R
    root.writePose(frame, recursive=True)

root.loadRestPose(recursive=True)
root = root.filter('Hips')[0].clearParent()
root.writeRestPose(recursive=False, keep=None)

root.loadPose(0, recursive=True)
for j, i, d in root.layout(): print(f"{j.PositionWorld} {j.Name}")

bvhio.writeHierarchy('test.bvh', root, 1/30, percision=5)

vec3(   -0.0205732,      1.23865,      -4.8484 )quat(     0.999084,    0.0262757,    0.0145151,   -0.0305009 )quat(            1,            0,            0,            0 )vec3(   -0.0205732,     0.110653,      -4.8484 )quat(     0.999084,    0.0262757,    0.0145151,   -0.0305009 )Hips
vec3(  -0.00892086,      1.42687,     -4.83866 )quat(     0.999145,    0.0215233,    0.0270672,   -0.0226474 )quat(            1,            0,            0,            0 )vec3( -6.14492e-08,  1.49012e-08, -6.62117e-09 )quat(     0.999879,  -0.00524651,    0.0126012,    0.0074494 )Spine
vec3(  -0.00391086,      1.53458,     -4.83415 )quat(     0.998596,    0.0062833,    0.0521882,  -0.00663647 )quat(            1,            0,            0,            0 )vec3(  6.26021e-09,  7.45058e-09,  1.10867e-08 )quat(      0.99944,   -0.0162174,    0.0251139,    0.0150316 )Chest
vec3( -0.000942836,      1.73104,     -4.82958 )quat(     0.998219,   -0.0465791,    0.0330735,   -0.0172145 )quat(            1,        